In [7]:
!pip install mne
!pip install pyprep
! pip install fooof

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 KB 13.0 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')
import os
import numpy as np
import mne
import matplotlib
import pathlib

from mne.preprocessing import ICA
from pyprep.find_noisy_channels import NoisyChannels
from pathlib import Path
from IPython import display

Mounted at /content/drive


In [4]:
data_path = '/content/drive/MyDrive/Intern/Sage/data/'
folder_list = os.listdir(data_path)
folder_list = [folder for folder in folder_list if folder.startswith('sub')]
# patient_list = [folder for folder in folder_list if folder[-4:-2]=='pa']
# control_list = [folder for folder in folder_list if folder[-4:-2]=='hc']

In [ ]:
prepro_idx = 0
delete = False
for folder_name in folder_list:
  print(prepro_idx)
  prepro_idx+=1
  file_name = folder_name + '_task-closed_eeg.vhdr'
  prepro_file_name = folder_name + '_task-closed_eeg_preprocessed.fif'
  folder_path = os.path.join(data_path,folder_name,'eeg')

  # check existed preprossed file
  file_path = os.path.join(folder_path, file_name)
  prepro_file_path = os.path.join(folder_path, prepro_file_name)

  if prepro_file_name in os.listdir(folder_path) and not delete:
    print(f"{file_name} has been preprocessed")
  elif prepro_file_name in os.listdir(folder_path) and delete:
    os.remove(prepro_file_path)
  else:
    # read raw file
    raw = mne.io.read_raw(file_path, preload=True)
    Is_fm = 1 if folder_name[-6:-4]=='FM' else 0 # FM data do not contain Ne and Ma channel

    raw = raw.set_channel_types({'LE':'eog','RE':'eog'}) 
    if not Is_fm: raw = raw.set_channel_types({'Ne':'emg','Ma':'emg'}) 

    # apply montage
    montage = mne.channels.make_standard_montage('standard_1020')
    raw = raw.set_montage(montage)
    # raw.plot_sensors(show_names=True)

    # resample data
    raw = raw.resample(250, npad='auto')

    # apply bandpass filter
    raw = raw.filter(l_freq=1., h_freq=100.)

    # apply notch filter
    raw = raw.notch_filter(50.)

    # find bad channels automatically
    raw_pyprep = NoisyChannels(raw, random_state=11)
    raw_pyprep.find_all_bads(ransac=False, channel_wise=False, max_chunk_size=None)
    raw.info['bads'] = raw_pyprep.get_bads()
    raw = raw.interpolate_bads(reset_bads=True)

    # apply ica
    ica = ICA(n_components=0.95, random_state=11, max_iter='auto')
    ica.fit(raw)
    eog_indices, eog_scores = ica.find_bads_eog(raw, threshold='auto')
    emg_indices, emg_scores = ica.find_bads_muscle(raw, threshold=0.5)
    ica.exclude = eog_indices
    ica.exclude = emg_indices
    ica.apply(raw)

    # apply average reference
    raw.set_eeg_reference(ref_channels='average')

    # save the preprossed file
    raw = raw.drop_channels(['LE','RE'])
    if not Is_fm: raw = raw.drop_channels(['Ne','Ma']) 
    raw.save(prepro_file_path, verbose=True, overwrite=True)
    display.clear_output(wait=True)

188
Extracting parameters from /content/drive/MyDrive/Intern/Sage/data/sub-NCCPpa41/eeg/sub-NCCPpa41_task-closed_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 300003  =      0.000 ...   300.003 secs...


<ipython-input-106-5d5c500f31e9>:20: RuntimeWarning: No coordinate information found for channels ['Ne', 'Ma']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw(file_path, preload=True)
<ipython-input-106-5d5c500f31e9>:20: RuntimeWarning: Not setting positions of 2 misc channels found in montage:
['Ne', 'Ma']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw(file_path, preload=True)
<ipython-input-106-5d5c500f31e9>:24: RuntimeWarning: The unit for channel(s) Ma, Ne has changed from NA to V.
  if not Is_fm: raw = raw.set_channel_types({'Ne':'emg','Ma':'emg'})


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 825 samples (3.300 sec)

Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwi

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  62 out of  62 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  62 out of  62 | elapsed:    0.2s finished


Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 825 samples (3.300 sec)



...

- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 sec)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 sec)

Effective window size : 1.024 (s)
Applying ICA to Raw instance


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


    Transforming to ICA space (8 components)
    Zeroing out 3 ICA components
    Projecting back using 62 PCA components
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Writing /content/drive/MyDrive/Intern/Sage/data/sub-NCCPpa41/eeg/sub-NCCPpa41_task-closed_eeg_preprocessed.fif


<ipython-input-106-5d5c500f31e9>:61: RuntimeWarning: This filename (/content/drive/MyDrive/Intern/Sage/data/sub-NCCPpa41/eeg/sub-NCCPpa41_task-closed_eeg_preprocessed.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(prepro_file_path, verbose=True, overwrite=True)


Closing /content/drive/MyDrive/Intern/Sage/data/sub-NCCPpa41/eeg/sub-NCCPpa41_task-closed_eeg_preprocessed.fif
[done]


In [5]:
folder_name = folder_list[53]
folder_path = os.path.join(data_path,folder_name,'eeg')
file_name = folder_name + '_task-closed_eeg_preprocessed.fif'
file_path = os.path.join(folder_path, file_name)
folder_name

'sub-FMhc19'

In [8]:
from fooof import FOOOF
raw = mne.io.read_raw_fif(file_path, preload=True)
evo_psds, freqs = (raw.compute_psd(method='multitaper', fmin=4, fmax=45)).get_data(return_freqs=True)
average_spec = evo_psds.mean(0)
fm = FOOOF()
fm.fit(freqs=freqs, power_spectrum=average_spec, freq_range=[4,45])

Opening raw data file /content/drive/MyDrive/Intern/Sage/data/sub-FMhc19/eeg/sub-FMhc19_task-closed_eeg_preprocessed.fif...
    Range : 0 ... 75284 =      0.000 ...   301.136 secs
Ready.
Reading 0 ... 75284  =      0.000 ...   301.136 secs...


<ipython-input-8-3c87608fe59b>:2: RuntimeWarning: This filename (/content/drive/MyDrive/Intern/Sage/data/sub-FMhc19/eeg/sub-FMhc19_task-closed_eeg_preprocessed.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(file_path, preload=True)


    Using multitaper spectrum estimation with 7 DPSS windows
